In [2]:
!pip install mysql-connector-python pandas bcrypt

In [3]:
import mysql.connector

# Function to establish a database connection
def get_db_connection():
    return mysql.connector.connect(
        host="localhost",
        user="root",    # Change if you have a different username
        password="1234",    # Add your MySQL password if required
        database="techfix_db"
    )

# Test database connection
try:
    conn = get_db_connection()
    print("Database connection successful!")
    conn.close()
except mysql.connector.Error as err:
    print(f"Error: {err}")

Database connection successful!


In [7]:
def create_users_table():
    conn = get_db_connection()
    cursor = conn.cursor()
    
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS users (
            id INT AUTO_INCREMENT PRIMARY KEY,
            full_name VARCHAR(100) NOT NULL,
            email VARCHAR(100) UNIQUE NOT NULL,
            phone VARCHAR(15) UNIQUE  NULL,
            location VARCHAR(15) UNIQUE  NULL,
            username VARCHAR(50) UNIQUE NOT NULL,
            joined_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            password VARCHAR(255) NOT NULL,
            user_type ENUM('Admin', 'Supplier') DEFAULT 'Supplier',
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        )
    """)

    cursor.execute("""
        CREATE TABLE IF NOT EXISTS quotations (
            quotation_id INT AUTO_INCREMENT PRIMARY KEY,
            requested_by VARCHAR(100) NOT NULL,
            items_needed VARCHAR(300) NOT NULL,       
            status ENUM('Pending', 'Completed') DEFAULT 'Pending',
            request_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        )
    """)

    cursor.execute("""
        CREATE TABLE IF NOT EXISTS orders (
    order_id INT AUTO_INCREMENT PRIMARY KEY,
    ordered_by VARCHAR(100) NOT NULL,
    items VARCHAR(300) NOT NULL,
    order_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    supplier VARCHAR(100) NOT NULL,
    order_status ENUM('Pending', 'Processing', 'Completed') DEFAULT 'Pending'
)

    """)

    cursor.execute("""
        CREATE TABLE IF NOT EXISTS inventory (
    id INT AUTO_INCREMENT PRIMARY KEY,
    part_number VARCHAR(50) NOT NULL,
    item_name VARCHAR(100) NOT NULL,
    stock_status VARCHAR(100) NOT NULL,
    reorder_level INT NOT NULL,
    unit_price DECIMAL(10,2) NULL,
    item_quantity INT NOT NULL,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);


    """)


    
    conn.commit()
    conn.close()
    print("✅ tables created successfully.")

# Run this function to create the table
create_users_table()

✅ tables created successfully.


In [44]:

import bcrypt

def register_user(full_name, email, username, password, user_type="Supplier",phone ="32322"):
    conn = get_db_connection()
    cursor = conn.cursor()
    
    # Hash the password before storing it
    hashed_password = bcrypt.hashpw(password.encode('utf-8'), bcrypt.gensalt())
    
    try:
        cursor.execute("""
            INSERT INTO users (full_name, email, username, password, user_type,phone)
            VALUES (%s, %s, %s, %s, %s,%s)
        """, (full_name, email, username, hashed_password, user_type, phone))
        
        conn.commit()
        print(f"✅ User {username} registered successfully.")
    except mysql.connector.Error as err:
        print(f"❌ Error: {err}")
    finally:
        conn.close()

# Example test registration
register_user("michel", "michel.doe@example.com", "michel", "123", "Supplier","09122111")


✅ User michel registered successfully.


In [42]:
import pandas as pd

def get_all_users():
    conn = get_db_connection()
    cursor = conn.cursor(dictionary=True)
    
    cursor.execute("SELECT id, full_name, email, username, user_type, created_at FROM users")
    users = cursor.fetchall()
    
    conn.close()
    
    return users

# Display all users
users_list = get_all_users()
df = pd.DataFrame(users_list)

# Display DataFrame properly in Jupyter Notebook
from IPython.display import display
display(df)


,id,full_name,email,username,user_type,created_at
0,1,michel,michel.doe@example.com,michel,Supplier,2025-03-18 12:54:36


In [36]:
def login_user(username, password):
    conn = get_db_connection()
    cursor = conn.cursor(dictionary=True)
    
    # Hash the input password for comparison
    hashed_password = bcrypt.hashpw(password.encode('utf-8'), bcrypt.gensalt())
    
    cursor.execute("""
        SELECT * FROM users WHERE username = %s AND password = %s
    """, (username, hashed_password))
    
    user = cursor.fetchone()
    conn.close()
    
    if user:
        print(f"✅ Login successful. Welcome, {user['full_name']}!")
        return user
    else:
        print("❌ Invalid username or password.")
        return None

# Example test login
login_user("michel", "123")


❌ Invalid username or password.
